In [12]:
import pandas as pd
import os
import shutil
import xmltodict
from PIL import Image

In [5]:
test_labels = pd.read_csv('./card_dataset/test_labels.csv')
train_labels = pd.read_csv('./card_dataset/train_labels.csv')

labels = pd.concat([test_labels, train_labels], ignore_index=True)
labels

,filename,width,height,class,xmin,ymin,xmax,ymax
0,cam_image2.jpg,960,540,king,312,30,485,249
1,cam_image2.jpg,960,540,king,514,24,694,245
2,cam_image2.jpg,960,540,king,305,263,489,519
3,cam_image2.jpg,960,540,king,515,267,704,523
4,cam_image4.jpg,960,540,jack,297,18,459,237
...,...,...,...,...,...,...,...,...
665,IMG_2700.JPG,378,504,ace,74,105,303,425
666,IMG_2701.JPG,378,504,jack,64,102,313,453
667,IMG_2702.JPG,378,504,nine,85,133,290,419
668,IMG_2703.JPG,378,504,ace,82,152,277,422


In [22]:
list(labels['class'].unique())


['king', 'jack', 'nine', 'ace', 'ten', 'queen']

In [23]:
CLASSES = {
    'ace' : 0,
    'king' : 1,
    'queen' : 2,
    'jack' : 3,
    'ten' : 4,
    'nine' : 5
}

In [15]:
def get_annotation(xml, img_path):
   with Image.open(img_path) as im:
      x_img_length, y_img_length = im.width, im.height

   elements = xml['annotation']['object']

   annotation_str = ''

   if isinstance(elements, dict):
      elements = [elements]
   for elem in elements:
      elem_class = CLASSES[elem['name']]

      x_box_length = float(elem['bndbox']['xmax']) - float(elem['bndbox']['xmin'])
      y_box_length = float(elem['bndbox']['ymax']) - float(elem['bndbox']['ymin'])
      x_box_center = float(elem['bndbox']['xmin']) + (x_box_length/2)
      y_box_center = float(elem['bndbox']['ymin']) + (y_box_length/2)

      x_box_length = x_box_length / x_img_length
      y_box_length = y_box_length / y_img_length

      x_box_center = x_box_center / x_img_length
      y_box_center = y_box_center / y_img_length

      if x_box_length > 1:
         x_box_length = 1.0
      if y_box_length > 1:
         y_box_length = 1.0
      if x_box_center > 1:
         x_box_center = 1.0
      if y_box_center > 1:
         y_box_center = 1.0

      annotation_str += f'{elem_class} {x_box_center} {y_box_center} {x_box_length} {y_box_length}\n'

   return annotation_str

In [16]:
xml_dir = './card_dataset/all/'
xml_names = os.listdir(xml_dir)
xml_paths = [f'{xml_dir}/{xml_name}' for xml_name in xml_names if xml_name.split('.')[-1] == 'xml']

In [18]:
xml_paths

['./card_dataset/all//cam_image1.xml',
 './card_dataset/all//cam_image10.xml',
 './card_dataset/all//cam_image11.xml',
 './card_dataset/all//cam_image12.xml',
 './card_dataset/all//cam_image13.xml',
 './card_dataset/all//cam_image14.xml',
 './card_dataset/all//cam_image15.xml',
 './card_dataset/all//cam_image16.xml',
 './card_dataset/all//cam_image17.xml',
 './card_dataset/all//cam_image18.xml',
 './card_dataset/all//cam_image19.xml',
 './card_dataset/all//cam_image2.xml',
 './card_dataset/all//cam_image20.xml',
 './card_dataset/all//cam_image21.xml',
 './card_dataset/all//cam_image22.xml',
 './card_dataset/all//cam_image23.xml',
 './card_dataset/all//cam_image24.xml',
 './card_dataset/all//cam_image25.xml',
 './card_dataset/all//cam_image26.xml',
 './card_dataset/all//cam_image27.xml',
 './card_dataset/all//cam_image28.xml',
 './card_dataset/all//cam_image29.xml',
 './card_dataset/all//cam_image3.xml',
 './card_dataset/all//cam_image30.xml',
 './card_dataset/all//cam_image31.xml',
 '.

In [24]:
for xml_path in xml_paths:
   with open(xml_path, 'r') as file:
      xml_string = ' '.join(file.readlines())

   xml_parsed = xmltodict.parse(xml_string)

   yolo_annotation = get_annotation(xml_parsed, xml_path.replace('xml', 'jpg'))
   with open(xml_path.replace('xml', 'txt'), 'w') as file:
      file.write(yolo_annotation)

In [55]:
file_path = './card_dataset/all/'
files = os.listdir(file_path)
txts = [file for file in files if file[-3:]=='txt']
jpgs = [file for file in files if file[-3:] == 'jpg' or file[-3:] == 'JPG']

In [56]:
len(jpgs)

363

In [58]:
for i in range(len(txts)):
    if i < len(txts)//10:
        dest = 'test'
    elif i >= len(txts)//10 and i < (len(txts)//10)*4:
        dest = 'val'
    else:
        dest = 'train'
    shutil.copy(file_path+txts[i], f'./data/labels/{dest}/{txts[i]}')
    shutil.copy(file_path+jpgs[i], f'./data/images/{dest}/{jpgs[i]}')

In [59]:
(len(txts)//10)*4

144